# Chapter 10 신경망 기계번역

## 10.1 기계번역

- seq2seq
- 어텐션

### 10.1.1 번역의 목표

$$\hat{e} = argmax P_{f \rightarrow e} (e|f)$$

#### 번역이 어려운 이유
- 언어의 모호성
- 문화의 차이

### 10.1.2 기계번역의 역사

#### 규칙 기반 기계번역(RBMT)

- 구조를 분석해서 얻은 규칙을 통해서 번역
- 한계
    - 규칙을 사람이 일일이 찾아야해서 많은 자원과 시간 소모
    - 번역 언어쌍을 확장할 때 매우 불리
    
#### 통게 기반 기계번역(SMT)

- 대량의 양방향 코퍼스에서 통계를 얻어내 번역 시스템 구성
- 언어쌍을 확장할 때 대부분의 알고리즘과 시스템은 유지되서 RBMT보다 유리

#### 딥러닝 이전의 신경망 기계번역

- 오토인코더 기반: 인코더 $\rightarrow$ 디코더
- 성능 매우 떨어짐

#### 딥러닝 이후의 신경망 기계번역

장점
1. end-to-end 모델  
    여러 모듈로 복잡하게 나눠지지 않고 하나의 모델로 번역을 해서 성능 극대화
2. 더 나은 언어 모델  
    n-gram의 단점이었던 희소성 문제를 NNLM으로 해결, 자연스러운 번역 결과 생성 강점
3. 훌륭한 문장 임베딩  
    임베딩 벡터 만들어내는 능력이 뛰어나기 때문에 노이즈나 희소성 문제 더 잘 대처

## 10.2 seq2seq

### 10.2.1 구조 소개

$P(Y|X;\theta)$를 최대로 하는 모델 파라미터를 찾고 사후 확률을 최대로 하는 $Y$를 찾는다.

$$\DeclareMathOperator*{\argmax}{argmax}$$
$$\hat{\theta} = \argmax_\theta P(Y|X;\theta) \ where \ X = \{x_1, x_2, \dots, x_n\}, Y = \{y_1, y_2, \dots, y_m\}$$
$$\hat{Y} = \argmax_{Y \in \mathcal{Y}}P(Y|X;\theta)$$

seq2seq는 **인코더**, **디코더**, **생성자**로 구성

#### 인코더

- 소스 문장으로부터 문장 임베딩 벡터를 만들어낸 후 $P(z|X)$를 모델링
- 주어진 문장을 매니폴드를 따라 차원 축소하여 해당 도메인의 잠재 공간의 어떤 하나의 점에 투영하는 작업
- 기계번역을 위한 문장 임베딩 벡터를 생성하려면 최대한 많은 정보를 갖고 있어야 함.

▶ 인코더 수식

$$h_t^{src} = RNN_{enc}(emb_{src}(x_t), h_{t-1}^{src})$$
$$H^{src} = [h_1^{src};h_2^{src};\dots;h_n^{src}]$$

▶ 전체 time-step 병렬 처리했을 때

$$H^{src} = RNN_{src}(emb_{src}(X), h_0^{src})$$

#### 디코더

조건부 신경망 언어 모델(CNNLM)

▶ seq2seq 모델 수식 time-step에 관해 풀었을 때
$$P_{\theta}(Y|X) = \prod_{t=1}^m P_{\theta}(y_t|X,y_{<t})$$
$$logP_{\theta}(Y|X) = logP_{\theta}(y_t|X,y_{<t})$$

- RNNLM에서 조건부 확률 변수 부분에 X가 추가된 모습
- 인코더의 결과인 문장 임베딩 벡터와 이전 time-step까지 번역하여 생성한 단어들에 기반하여 현재 time-step의 단어 생성

▶ 디코더 수식
$$h_t^{tgt} = RNN_{dec}(emb_{tgt}(y_{t-1}), h_{t-1}^{tgt})$$
$$where \ h_0^{tgt} = h_n^{tgt} and y_0 = BOS$$

#### 생성자

디코더에서 각 time-step별로 결과 벡터 $h_t^{tgt}$를 받아 softmax를 계산하여, 각 타깃 언어의 단어별 확률값을 반환

▶ 생성자 수식

$$\hat{y}_t = softmax(linear_{hs \to |V_{tgt}|}(h_t^{tgt})) and \hat{y}_m = EOS$$
$$where \ hs \ is \ hidden \ size \ of \ RNN, \ and \ |V_{tgt}| \ is \ size \ of \ output \ vocabulary$$

### 10.2.2. seq2seq의 활용 분야

기계번역, 챗봇, 요약, 기타 자연어 처리, 음성 인식, 독순술, 이미지 캡셔닝 등

### 10.2.3 한계점

#### 장기 기억력

정보를 압축하는데 한계가 있다보니까 시퀀스가 길어질수록 성능이 떨어진다.

#### 구조 정보의 부재

현재는 문장을 단순히 시퀀스 데이터로 다루고 있지만, 나중에는 구조 정보도 필요할 것이다.

#### 챗봇 또는 QA봇

번역이나 요약 문제의 경우에는 새롭게 추가되는 정보가 거의 없어서 잘 수행했지만, 대화의 경우에는 지식이나 맥락 등 추가되는 정보가 있기 때문에 한계가 있다.

### 10.2.4. 파이토치 예제 코드

#### 인코더 클래스

In [3]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    
    def __init__(self, word_vec_dim, hidden_size, n_layers=4, dropout_p=.2):
        super(Encoder, self).__init__()
        
        self.rnn = nn.LSTM(word_vec_dim,
                           int(hidden_size / 2),
                           num_layers=n_layers,
                           dropout=dropout_p,
                           bidirectional=True,
                           batch_first=True
                           )
        
    def forward(self, emb):
        if isinstance(emb, tuple):
            x, lengths = emb
            x = pack(x, lenghts.tolist(), batch_first=True)
        
        else:
            x = emb
            
        y, h = self.rnn(x)
    
        if isinstance(emb, tuple):
            y, _ = unpack(y, batch_first=True)
        
        return y, h

####  pack_padded_sequence 함수

In [13]:
a = [torch.tensor([1, 2, 3]), torch.tensor([3, 4]), torch.tensor([1])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)

In [14]:
a

[tensor([1, 2, 3]), tensor([3, 4]), tensor([1])]

In [15]:
b

tensor([[1, 2, 3],
        [3, 4, 0],
        [1, 0, 0]])

In [20]:
torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True, lengths=[3, 2, 1])

PackedSequence(data=tensor([1, 3, 1, 2, 4, 3]), batch_sizes=tensor([3, 2, 1]), sorted_indices=None, unsorted_indices=None)

#### 디코더 클래스

어텐션 이후에

#### 생성자 클래스

In [21]:
class Generator(nn.Module):
    
    def __init__(self, hidden_size, output_size):
        super(Generator, self).__init__()
        
        self.output = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self, x):
        y = self.softmax(self.output(x))
        
        return y

#### 전체 seq2seq 클래스

어텐션 이후

#### 손실 함수

교차 엔트로피 함수

실제 구현에서는 **softmax + 교차 엔트로피**보다 **logsoftmax + 음의 로그 가능도(NLL)**를 사용한다.

In [ ]:
loss_weight = torch.ones(output_size)
loss_weight[data_loader.PAD] = 0.

crit = nn.NLLLoss(weight=loss_weight,
                  reduction='sum',
                  )

In [23]:
def _get_loss(self, y_hat, y, crit=None):
    crit = self.crit if crit is None else crit
    loss = crit(y_hat.contiguous().view(-1, y_hat.size(-1)),
                y.contiguous().view(-1))
    
    return loss

```contiguous()```: 특정 축 잘라내는 함수

In [24]:
x = torch.arange(1*2*2*6).view(-1,2,2,6)
print(x)

tensor([[[[ 0,  1,  2,  3,  4,  5],
          [ 6,  7,  8,  9, 10, 11]],

         [[12, 13, 14, 15, 16, 17],
          [18, 19, 20, 21, 22, 23]]]])


In [25]:
x = x.view(-1,6)
print(x)

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]])


In [27]:
x = x[:,:4].contiguous()
print(x)

tensor([[ 0,  1,  2,  3],
        [ 6,  7,  8,  9],
        [12, 13, 14, 15],
        [18, 19, 20, 21]])


## 10.3 어텐션

### 10.3.1 어텐션 이해하기

쿼리와 비슷한 값을 가진 키를 찾아서 값을 얻는 과정

### 10.3.2 key-value 함수

In [28]:
dic = {'computer': 9, 'dog': 2, 'cat': 3}

In [30]:
def key_value_func(query):
    weights = []
    
    for key in dic.keys():
        weights += [is_same(key, query)]
        
    weight_sum = sum(weights)
    for i,w in enumerate(weights):
        weights[i] = weights[i] / weight_sum
        
    answer = 0
    
    for weight, value in zip(weights, dic.values()):
        answer += weight *  value
        
    return answer

In [31]:
def is_same(key, query):
    if key == query:
        return 1.
    else:
        return 0.

In [35]:
for k in dic.keys():
    print(is_same(k, "computer"))

1.0
0.0
0.0


In [36]:
print(key_value_func("computer"))

9.0


### 10.3.3 연속적인 key-value 함수

...

### 10.3.4 연속적인 key-value 벡터 함수

...

### 10.3.5 기계번역에서의 어텐션

인코더의 각 time-step별 출력을 키와 밸류로 삼고, 현재 time-step의 디코더 출력을 쿼리로 삼아 어텐션 계산

▶ 어텐션을 추가한 seq2seq 수식

$$w = softmax(h_t^{tgtT}W \cdot H^{src})$$
$$c = H^{src} \cdot w \  and c \ is \ a\ context \ vector$$
$$\tilde{h}_t^{tgt} = tanh(linear_{2 \times hs \to hs}([h_t^{tgt};c]))$$
$$\hat{y}_t = softmax(linear_{hs \to |V_{tgt}|}(\tilde{h}_t^{tgt}))$$
$$where \ hs \ is \ hidden \ size \ of \ RNN, \ and \ |V_{tgt}| \ is \ size \ of \ output \ vocabulary$$

- 원하는 정보를 어텐션을 통해 인코더에서 획득한 후, 해당 정보를 디코더의 출력과 이어붙여 tanh를 취한 후, softmax 계산을 통해 다음 time-step의 입력이 되는 $\hat y_t$을 구한다.

<img src="https://blobscdn.gitbook.com/v0/b/gitbook-28427.appspot.com/o/assets%2F-LFzjkZt6ljMBd4YGVCn%2F-LX4NCwvtTXyg_vzXhF0%2F-LX4NFi6LreOTrZ3dcWF%2F10-03-01.png?generation=1548425967953999&alt=media" width="80%">

#### 선형 변환

소스 언어와 대상 언어가 애초부터 다르기 때문에 선형 관계가 있다고 가정하고 내적을 수행하기 전에 선형 변환을 해준다. $W$는 선형 변환을 위한 신경망 가중치 파라미터이다.

어텐션이 필요한 이유는 문장이 길어질수록 은닉 상태만으로 모든 정보를 완벽하게 전달할 수 없는 문제 때문이다. 따라서 디코더의 time-step마다 현재 디코더의 은닉 상태에 따라 필요한 인코더의 정보에 접근해서 사용하겠다는 것이다.

선형 변환을 배우는 것 자체를 어텐션이라고 표현할 수 있다. 이는 디코더의 현재 상태에 따라 필요한 쿼리를 만들어내고, 인코더의 key 값들과 비교해서 가중합을 하는 과정이기 때문이다.

선형 변환을 위한 가중치 파라미터가 훈련되면, 디코더의 상태 자체도 선형 변환에 좋은 방향으로 RNN이 동작할 것이다.

#### 선형 변환 결과

어텐션을 적용한 것이 성능이 더 좋았고, 특히 문장이 길어질수록 성능이 떨어지는 점이 덜해졌다.

### 10.3.6 파이토치 예제 코드

```torch.bmm()```: 배치 행렬 곱을 수행하는 함수, 뒤의 두개의 차원에 대해 행렬 곱을 수행하기 때문에 이 부분의 차원은 행렬 곱에 적합한 크기여야 하고, 앞의 차원들은 배치로 취급해서 같은 크기여야 한다.

In [39]:
import torch

x = torch.arange(80).view(10, 2, 4)
y = torch.arange(240).view(10, 4, 6)

x.size(), y.size()

(torch.Size([10, 2, 4]), torch.Size([10, 4, 6]))

In [40]:
# |x| = (10, 2, 4)
# |y| = (10, 4, 6)

z = torch.bmm(x, y)
# |z| = (10, 2, 6)

In [42]:
z.size()

torch.Size([10, 2, 6])

#### 어텐션 클래스

선형 변환을 위한 가중치 파라미터를 편향이 없는 선형 계층으로 대체

In [43]:
class Attention(nn.Module):
    
    def __init(self, hidden_size):
        super(Attention, self).__init__()
        
        self.linear = nn.Linear(hidden_size, hidden_size, bias=False)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, h_src, h_t_tgt, mask=None):
        query = self.linear(h_t_tgt.squeeze(1)).unsqueeze(-1)
        
        weight = torch.bmm(h_src, query).squeeze(-1)
        
        if mask is not None:
            weight.masked_fill_(mask, -float('inf'))
        weight = self.softmax(weight)
        
        context_vector = torch.bmm(weight.unsqueeze(1), h_src)
        
        return context_vector

## 10.4 input feeding

- softmax 이전의 값을 임베딩 벡터에 이어붙여 디코더의 RNN의 입력으로 넣는다.
- 이유: softmax를 해서 샘플링하는 과정에서 많은 정보가 손실되기 때문

▶ 어텐션과 input feeding이 추가된 seq2seq 수식

$$h_t^{src} = RNN_{enc}(emb_{src}(x_t),h_{t-1}^{src})$$
$$H^{src} = [h_1^{src};h_2^{src};\dots;h_n^{src}]$$
$$h_t^{tgt} = RNN_{dec}([emb_{tgt}(y_{t-1});\tilde{h}_{t-1}^{tgt}],h_{t-1}^{tgt}) \ where \ h_0^{tgt} = h_n^{src} \ and \ y_0 = BOS$$
$$w = softmax(h_t^{tgtT}W \cdot H^{src})$$
$$c = H^{src} \cdot w \ and \ c \ is \ a \ context \ vector$$
$$h_t^{tgt} = tanh(linear_{2hs \to hs} ([h_t^{tgt};c]))$$
$$\hat{y}_t = softmax(linear_{hs \to |V_{tgt}|} (\tilde {h}_t^{tgt}))$$
$$where \ hs \ is \ hidden \ size \ of \ RNN, \ and \ |V_{tgt}| \ is \ size \ of \ output \ vocabulary$$

### 10.4.1 단점

- 훈련 속도 저하: 디코더 RNN의 입력으로 이전 time-step의 결과($\tilde{h}_t^{tgt}$)가 필요해서 time-step 별로 순차적으로 처리
- 하지만 추론 과정에서는 병렬 처리가 거의 불가능하기 때문에 크게 부각되진 않는다.

### 10.4.2 성능 개선

어텐션과 input feeding을 사용해서 기본 모델보다 더 나은 성능 보임.

### 10.4.3 구현 관점에서 바라보기

※ 각 텐서의 크기 생각해보기

1. 소스 문장과 타깃 문장으로 이루어진 병렬 코퍼스

$$B = \{X_i, Y_i\}_{i=1}^N$$
$$ where \ X = \{x_1, \dots, x_n\}, Y = \{y_1, \dots, y_m\}$$

2. 인코더에 넣고 피드포워드하는 과정

    ▶ 1 time-step의 경우
$$h_t^{src} = RNN_{enc}(emb_{src}(x_t),h_{t-1}^{src})$$

    ▶ 병렬 처리
$$H^{src} = [h_1^{src};h_2^{src};\dots;h_n^{src}] = RNN_{enc}(emb_{src}(X),h_0^{src})$$


3. 디코더의 피드포워드

$$h_t^{tgt} = RNN_{dec}([emb_{tgt}(y_{t-1});\tilde{h}_{t-1}^{tgt}], h_{t-1}^{tgt})$$
$$where \ h_0^{tgt} = h_n^{src} \ and \ y_0 = BOS$$

4. 어텐션 가중치

$$w = softmax(H^{src} \cdot (h_t^{tgt} \cdot W))$$

5. 어텐션 가중치에 따라 가중합 구하는 과정

$$c = w \cdot H^{src}$$

6. input-feeding

$$\tilde{h}_t^{tgt} = tanh(linear_{2hs \to hs}([h_t^{tgt};c]))$$

7. 소프트맥스 계층을 통과시켜 확률값 구하기

$$\DeclareMathOperator*{\argmax}{argmax}$$
$$P(y_t|X,y_{<t};\theta) = softmax(linear_{hs \to |V_tgt|}(\tilde{h}_t^{tgt}))$$
$$\hat{y}_t = \argmax_{y \in \mathcal{Y}} P(y_t|X,y_{<t};\theta)$$

8. 손실함수

$$\mathcal{L}(\theta) = - \frac 1 N \sum_{i=1}^N y_t^i \cdot logP(y_t|X_i,y_{<t}^t;\theta)$$

9. 파라미터 업데이트

$$\theta \leftarrow \theta - \gamma \nabla_{\theta}\mathcal{L}_{\theta}(\hat{Y}, Y)$$

### 10.4.4 파이토치 예제 코드

#### 어텐션 클래스

In [1]:
import torch
import torch.nn as nn

In [3]:
class Attention(nn.Module):
    
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        
        self.linear = nn.Linear(hidden_size, hidden_size, bias=False)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, h_src, h_t_tgt, mask=None):
        query = self.linear(h_t_tgt.squeeze(1)).unsqueeze(-1)
        
        weight = torch.bmm(h_src, query).squeeze(-1)
        
        if mask is not None:
            weight.masked_fill_(mask, -float('inf'))
        weight = self.softmax(weight)
        
        context_vector = torch.bmm(weight.unsqueeze(1), h_src)
        
        return context_vector

#### 어텐션을 위한 마스크 생성

- 미니배치의 크기는 가장 긴 문장의 크기에 좌우됨
- 짧은 문장들은 문장의 종료 후 패딩으로 채워짐
- 이런 미니배치가 어텐션 연산을 수행하면 단어가 존재하지 않는 곳인데 디코더에게 쓸데없이 정보를 넘겨주게 됨
- 따라서 이 부분의 어텐션 가중치를 다시 0으로 만들어주는 작업 필요
- 마스크 부분을 음의 무한대 값을 줘서 softmax를 통과했을 때 0이 되도록 한다.

In [5]:
# seq2seq 클래스 내부에 정의된 마스크 생성 함수
def generate_mask(self, x, length):
    mask = []
    
    max_length = max(length)
    for l in length:
        if max_length - l > 0:
            mask += [torch.cat([x.new_ones(1, l).zero_(),
                                x.new_ones(1, (max_length-1))
                               ], dim=-1)]
        else:
            mask += [x.new_ones(1, l).zero_()]
            
    mask = torch.cat(mask, dim=0).byte()
    
    return mask

#### 인코더 클래스

In [6]:
class Encoder(nn.Module):
    
    def __init__(self, word_vec_dim, hidden_size, n_layers=4, dropout_0=.2):
        super(Encoder, self).__init__()
        
        self.rnn = nn.LSTM(word_vec_dim,
                           int(hidden_size/2),
                           num_layers=n_layers,
                           dropout=dropout_p,
                           bidirectional=True,
                           batch_first=True
                          )
        
    def forward(self, emb):
        if isinstance(emb, tuple):
            x, lengths = emb
            x = pack(x, lengths.tolist(), batch_first=True)
            
        else:
            x = emb
            
        y, h = self.rnn(x)
        
        if isinstance(emb, tuple):
            y, _ = unpack(y, batch_first=True)
            
        return y, h

#### 디코더 클래스

In [7]:
class Decoder(nn.Module):
    
    def __init__(self, word_vec_dim, hidden_size, n_layers=4, dropout=.2):
        super(Decoder, self).__init__()
        
        self.rnn = nn.LSTM(word_vec_dim + hidden_size,
                           hidden_size,
                           num_layers=n_layers,
                           dropout=dropout_p,
                           bidirectional=False,
                           batch_first=True
                          )
        
    def forward(self, emb_t, h_t_1_tilde, h_t_1):
        batch_size = emb_t.size(0)
        hidden_size = h_t_1[0].size(-1)
        
        if h_t_1_tilde is None:
            h_t_1_tilde = emb_t.new(batch_size, 1, hidden_size).zero_()
            
        x = torch.cat([emb_t, h_t_1_tilde], dim=-1)
        
        y, h = self.rnn(x, h_t_1)
        
        return y, h

#### 생성자 클래스

In [8]:
class Generator(nn.Module):
    
    def __init__(self, hidden_size, output_size):
        super(Generator, self).__init__()
        
        self.output = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self, x):
        y = self.softmax(self.output(x))
        
        return y

#### seq2seq 클래스

In [9]:
class Seq2Seq(nn.Module):
    
    def __init__(self,
                 input_size,
                 word_vec_dim,
                 hidden_size,
                 output_size,
                 n_layers=4,
                 dropout_p=.2
                 ):
        self.input_size = input_size
        self.word_vec_dim = word_vec_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        super(Seq2Seq, self).__init__()
        
        self.emb_src = nn.Embedding(input_size, word_vec_dim)
        self.emb_dec = nn.Embedding(ouput_size, word_vec_dim)
        
        self.encoder = Encoder(word_vec_dim,
                               hidden_size,
                               n_layers=n_layers,
                               dropout_p=dropout_p
                               )
        self.decoder = Decoder(word_vec_dim,
                               hidden_size,
                               n_layers=n_layers,
                               dropout_p=dropout_p
                               )
        self.attn = Attention(hidden_size)
        
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.tanh = nn.Tanh()
        self.generator = Generator(hidden_size, output_size)

#### 인코더의 은닉 상태를 디코더의 은닉 상태로 변환하기

인코더는 양방향 LSTM을 사용했기 때문에 텐서의 위치를 조작하여 디코더의 은닉 상태에 알맞은 형태로 변환해야 한다.

In [11]:
def merge_encoder_hiddens(self, encoder_hiddens):
    new_hiddens = []
    new_cells = []
    
    hiddens, cells = encoder_hiddens
    
    for i in range(0, hiddens.size(0), 2):
        new_hiddens += [torch.cat([hiddens[i], hiddens[i + 1]], dim=-1)]
        new_cells += [torch.cat([cells[i], cells[i + 1]], dim=-1)]
        
    new_hiddens, new_cells = torch.stack(new_hiddens), torch.stack(new_cells)
    
    return (new_hiddens, new_cells)

In [ ]:
# 반복문을 사용하지 않는 방법
h_0_tgt, c_0_tgt = h_0_tgt
h_0_tgt = h_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                    -1,
                                                    self.hidden_size
                                                    ).transpose(0, 1).contiguous()
c_0_tgt = c_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                    -1,
                                                    self.hidden_size
                                                    ).transpose(0, 1).contiguous()

#### 포워드 함수 정의

seq2seq에서 인코더는 모든 time-step을 한번에 연산하지만, 디코더는 input feeding 때문에 반복문을 사용해서 타깃 문장의 길이만큼 time-step 수행

In [1]:
def forward(self, src, tgt):
    batch_size = tgt.size(0)
    
    mask = None
    x_length = None
    if isinstance(src, tuple):
        x, x_length = src
        mask = self.generate_mask(x, x_length)
    
    else:
        x = src
    
    if isinstance(tgt, tuple):
        tgt = tgt[0]
        
    emb_src = self.emb_src(x)
    
    h_src, h_0_tgt = self.encoder((emb_src, x_length))
    
    h_0_tgt, c_0_tgt = h_0_tgt
    h_0_tgt = h_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                        -1,
                                                        self.hidden_size
                                                        ).transpose(0, 1).contiguous()
    c_0_tgt = c_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                        -1,
                                                        self.hidden_size
                                                        ).transpose(0, 1).contiguous()
    
    h_0_tgt = (h_0_tgt, c_0_tgt)
    
    emb_tgt = self.emb_dec(tgt)
    
    h_tilde = []
    h_t_tilde = None
    decoder_hidden = h_0_tgt
    
    for t in range(tgt.size(1)):
        emb_t = emb_tgt[:, t, :].unsqueeze(1)
        
        decoder_output, decoder_hidden = self.decoder(emb_t,
                                                      h_t_tilde,
                                                      decoder_hidden
                                                      )
        
        context_vector = self.attn(h_src, decoder_output, mask)
        
        h_t_tilde = self.tanh(self.concat(torch.cat([decoder_output,
                                                     context_vector
                                                     ], dim=-1)))
        
        h_tilde += [h_t_tilde]
        
    h_tilde = torch.cat(h_tilde, dim=1)
    
    y_hat = self.generator(h_tilde)
    
    return y_hat

## 10.5 자귀회귀 속성과 teacher forcing 훈련 방법

seq2seq의 훈련과 추론은 다르다.

### 10.5.1 자기회귀 속성

- 자기회귀(autogressive): 과거 자신의 값을 참조하여 현재의 값을 추론하는 특징

▶ 신경망 기계번역 수식
$$\hat{Y} = \argmax_{Y \in \mathcal{Y}}P(Y|X) = \argmax_{Y \in \mathcal{Y}}\prod_{i=1}^nP(y_i|X, \hat{y}_{<i})$$ 
$$or$$
$$\hat{y}_t = \argmax_{Y \in \mathcal{Y}}P(y_t|X,\hat{y}_{<t};\theta)$$

학습 과정에서는 이미 정답을 알고 있고, 예측값과 정답과의 차이를 통해 학습하므로 자기회귀 속성을 유지할 수 없다.

### 10.5.2 teachar forcing 훈련 방식

<img src="https://blobscdn.gitbook.com/v0/b/gitbook-28427.appspot.com/o/assets%2F-LFzjkZt6ljMBd4YGVCn%2F-LX4NCwvtTXyg_vzXhF0%2F-LX4NFdMA7NCZp2QYb4z%2F10-05-01.png?generation=1548425965734583&alt=media" width="80%">

훈련할 때는 이전 time-step의 출력을 현재 time-step의 입력으로 넣어줄 수 없다. 하지만 추론할 때는 정답을 모르기 때문에 이전 time-step에서 나온 예측값을 입력으로 넣어준다. 이렇게 훈련하는 방식을 **teacher forcing**이라고 한다.

훈련할 때는 입력값이 정해져 있으므로 input feeding이 없는 디코더는 모든 time-step을 한번에 수행할 수 있다.

$$H_{tgt} = RNN_{dec}(emb_{dec}([BOS;Y[:-1]]),h_n^{src})$$

input feeding은 이전 time-step의 softmax 이전 계층의 값을 단어 임베딩 벡터와 함께 받아야하므로 모든 time-step을 한번에 처리할 수 없다.

$$h_t^{tgt} = RNN_{dec}([emb_{tgt}(y_{t-1});\tilde{h}_{t-1}^{tgt})$$

언어 모델의 경우 퍼플렉서티가 문장의 확률과 직접적으로 연관이 있기 때문에 문제가 별로되지 않지만, 기계번역에서는 문제

## 10.6 탐색(추론)

X만 추어진 상태에서 추론하는 방법

### 10.6.1 샘플링

- 예측값을 softmax 계층의 확률 분포대로 샘플링
- 최종적으로 EOS가 나올 때까지
- 동일 입력에서 매번 다른 출력 결과 나옴

### 10.6.2 탐욕 탐색 알고리즘 활용

소프트맥스 계층에서 가장 확률값이 큰 인덱스를 사용

### 10.6.3 파이토치 예제코드

In [3]:
def search(self, src, is_greedy=True, max_length=255):
    mask, x_length = None, None
    
    if isinstance(src, tuple):
        x, x_length = src
        mask = self.generate_mask(x, x_length)
    else:
        x = src
    batch_size = x.size(0)
    
    emb_src = self.emb_src(x)
    h_src, h_0_tgt = self.encoder((emb_src, x_length))
    h_0_tgt, c_0_tgt = h_0_tgt
    h_0_tgt = h_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                        -1,
                                                        self.hidden_size
                                                        ).transpose(0, 1).contiguous()
    c_0_tgt = c_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                        -1,
                                                        self.hidden_size
                                                        ).transpose(0, 1).contiguous()
    h_0_tgt = (h_0_tgt, c_0_tgt)
    
    y = x.new(batch_size, 1).zero_() + data_loader.BOS
    is_undone = x.new_ones(batch_size, 1).float()
    decoder_hidden = h_0_tgt
    h_t_tilde, y_hats, indice = None, [], []
    
    while is_undone.sum() > 0 and len(indice) < max_length:
        emb_t = self.emb_dec(y)
        
        decoder_output, decoder_hidden = self.decoder(emb_t,
                                                      h_t_tilde,
                                                      decoder_hidden
                                                      )
        context_vector = self.attn(h_src, decoder_output, mask)
        h_t_tilde = self.tanh(self.concat(torch.cat([decoder_output,
                                                     context_vector
                                                     ], dim=-1)))
        y_hat = self.generator(h_t_tilde)
        y_hats += [y_hat]
        
        if is_greedy:
            y = torch.toqk(y_hat, 1, dim=-1)[1].squeez(-1)
        else:
            y = torch.multinomial(y_hat.exp().view(batch_size, -1), 1)
            
        y = y.masked_fill_((1. - is_undone).byte(), data_loader.PAD)
        is_undone = is_undone * torch.ne(y, data_loader.EOS).float()
        indice += [y]
    
    y_hats = torch.cat(y_hats, dim=1)
    indice = torch.cat(indice, dim=-1)
    
    return y_hats, indice

### 10.6.4 빔서치

- k개의 후보를 더 추적해서 최적 해에 좀 더 가까워지도록
- k x |V| 개의 softmax 결과 중 최고 누적확률 k개를 뽑아 다음 time-step에 넘김
    - 현재 time-step까지의 로그 확률에 대한 합 추적하고 있어야 함
    - 이전 time-step에서 뽑인 k개에 대해 게산한 현재 time-step의 모든 결과물 중에서 최고 누적확률 k개를 다시 뽑는다.
- k개의 미니배치를 만들어 수행하면 병렬 처리 가능
- 보통 빔 크기는 10 이하

#### 구현 관점에서 바라보기

- EOS가 k개 나올 때까지 수행

#### 길이 패널티

- 현재 time-step까지의 확률의 모든 곱으로 구해지므로 문장이 길어질수록 확률이 낮아지는 단점
- 예측한 문장의 길이에 따른 페널티를 주어 짧은 문장이 긴 문장을 제치고 선택되는 문제 방지

$$log \tilde P(\hat Y|X) = logP(\hat Y|X) \times penalty$$
$$penalty = \frac {(1+length)^\alpha} {(1 + \beta)^\alpha}$$

In [8]:
alpha = 1.2
beta = 5

for l in range(1, 10):
    penalty = ((1 + l)**alpha)/((1 + beta)**alpha)
    print(penalty)

0.26758052058674353
0.43527528164806206
0.6147386076544852
0.8034937533355226
1.0
1.203195357557136
1.4122984547317496
1.6267076567965477
1.8459439054138165


### 10.6.5 파이토치 예제 코드

#### SingleBeamSearchSpace 클래스

#### 클래스 선언 및 초기화

In [5]:
from operator import itemgetter

import torch
import torch.nn as nn
# import data_loader

In [6]:
LENGTH_PENALTY = 1.2
MIN_LENGTH = 5

In [ ]:
class SingleBeamSearchSpace():
    
    def __init__(self,
                 hidden,
                 h_t_tilde=None,
                 beam_size=5,
                 max_length=255):
        self.beam_size = beam_size
        self.max_length = max_length
        
        super(SingleBeamSearchSpace, self).__init__()
        
        self.device = hidden[0].device
        self.word_indice = [torch.LongTensor(beam_size).zero_().to(self.device) + data_loader.BOS]
        self.prev_beam_indice = [torch.LongTensor(beam_size).zero_().to(self.device) - 1]
        self.cumulative_probs = [torch.FloatTensor([.0] + [-float('inf')] * (beam_size - 1)).to(self.device)]
        self.masks = [torch.ByteTensor(beam_size).zero_().to(self.device)]
        
        self.prev_hidden = torch.cat([hidden[0]] * beam_size, dim=1)
        self.prev_cell = torch.cat([hidden[1]] * beam_size, dim=1)
        
        self.prev_h_t_tilde = torch.cat([h_t_tilde] * beam_size,
                                        dim=0
                                        ) if h_t_tilde is not None else None
        
        self.current_time_step = 0
        self.done_cnt = 0
        
    # 길이 페널티 구현
    def get_length_penalty(self,
                           length,
                           alpha=LENGTH_PENALTY,
                           min_length=MIN_LENGTH
                           ):
        p = (1 + length) ** alpha / (1 + min_length) ** alpha
        
        return p
    
    # 디코딩 작업 종료 체크
    def is_done(self):
        if self.don_cnt >= self.beam_size:
            return 1
        return 0
    
    # 가짜 미니배치의 일부 만들기
    def get_batch(self):
        y_hat = self.word_indice[-1].unsqueeze(-1)
        hidden = (self.prev_hidden, self.prev_call)
        h_t_tilde = self.prev_h_t_tilde
        
        return y_hat, hidden, h_t_tilde
    
    # 최고 누적확률 k개 고르기
    def collect_result(self, y_hat, hidden, h_t_tilde):
        output_size = y_hat.size(-1)
        
        self.current_time_step += 1
        cumulative_prob = y_hat + \
            self.cumulative_probs[-1].masked_fill_(self.masks[-1], -float('inf')) \
            .view(-1, 1, 1).expand(self.beam_size, 1, output_size)
        top_log_prob, top_indice = torch.topk(cumulative_prob.view(-1),
                                              self.beam_size,
                                              dim=-1
                                              )
        
        self.word_indice += [top_indice.fmod(output_size)]
        self.prev_beam_indice += [top_indice.div(output_size).long()]
        
        self.cumulative_probs += [top_log_prob]
        self.masks += [torch.eq(self.word_indice[-1],
                                data_loader.EOS)]
        self.done_cnt += self.masks[-1].float().sum()
        
        self.prev_hidden = torch.index_select(hidden[0],
                                              dim=1,
                                              index=self.prev_beam_indice[-1]
                                              ).contiguous()
        self.prev_cell = torch.index_select(hidden[1],
                                            dim=1,
                                            index=self.prev_beam_indice[-1]
                                            ).contiguous()
        self.prev_h_t_tilde = torch.index_select(h_t_tilde,
                                                 dim=0,
                                                 index=self.prev_beam_indice[-1]
                                                 ).contiguous()
        
    # 결괏값 정리
    def get_n_best(self, n=1):
        sentences, probs, founds = [], [], []
        
        for t in range(len(self.word_indice)):
            for b in range(self.beam_size):
                if self.masks[t][b] == 1:
                    probs += [self.cumulative_probs[t][b] / self.get_length_penalty(t)]
                    founds += [(t, b)]
        
        for b in range(self.beam_size):
            if self.cumulative_probs[-1][b] != -float('inf'):
                if not (len(self.cumulative_probs) - 1, b) in founds:
                    probs += [self.cumulative_probs[-1][b]]
                    founds += [(t, b)]
                    
        sorted_founds_with_probs = sorted(zip(founds, probs),
                                          key=itemgetter(1),
                                          reverse=True
                                          )[:n]
        prob = []
        
        for (end_index, b), prob in sorted_founds_with_probs:
            sentence = []
            for t in range(end_index, 0, -1):
                sentence = [self.word_indice[t][b]] + sentence
                b = self.prev_beam_indice[t][b]
                
            sentences += [sentence]
            probs += [prob]
            
        return sentences, probs
    

In [18]:
# 병렬 빔서치 수행 함수
def batch_beam_search(self, src, beam_size=5, max_length=255, n_best=1):
    mask, x_length = None, None
    
    if isinstance(src, tuple):
        x, x_length = src
        mask = self.generate_mask(x, x_length)
    else:
        x = src
    batch_size = x.size(0)
    
    emb_src = self.emb_src(x)
    h_src, h_0_tgt = self.encoder((emb_src, x_length))
    h_0_tgt, c_0_tgt = h_0_tgt
    h_0_tgt = h_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                        -1,
                                                        self.hidden_size
                                                        ).transpose(0, 1).contiguous()
    c_0_tgt = c_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                        -1,
                                                        self.hidden_size
                                                        ).transpose(0, 1).contiguous()
    
    h_0_tgt = (h_0_tgt, c_0_tgt)
    
    spaces = [SingleBeamSearchSpace((h_0_tgt[0][:, i, :].unsqueeze(1),
                                     h_0_tgt[1][:, i, :].unsqueeze(1)
                                     ),
                                    None,
                                    beam_size,
                                    max_length=max_length
                                    ) for i in range(batch_size)]
    done_cnt = [space.is_done() for space in spaces]
    
    length = 0
    
    while sum(done_cnt) < batch_size and length <= max_length:
        fab_input, fab_hidden, fab_cell, fab_h_t_tilde = [], [], [], []
        fab_h_src, fab_mask = [], []
        
        for i, space in enumerate(spaces):
            if space.is_done() == 0:
                y_hat_, (hidden_, cell_), h_t_tilde_ = space.get_batch()
                fab_input += [y_hat_]
                fab_hidden += [hidden_]
                fab_cell += [cell_]
                if h_t_tilde_ is not None:
                    fab_h_t_tilde += [h_t_tilde_]
                else:
                    fab_h_t_tilde = None
                    
                fab_h_src += [h_src[i, :, :]] * beam_size
                fab_mask += [mask[i, :]] * beam_size
            
            fab_input = torch.cat(fab_input, dim=0)
            fab_hidden = torch.cat(fab_hidden, dim=1)
            fab_cell = torch.cat(fab_cell, dim=1)
            if fab_h_t_tilde is not None:
                fab_h_t_tilde = torch.cat(fab_h_t_tilde, dim=0)
            fab_h_src = torch.stack(fab_h_src)
            fab_mask = torch.stack(fab_mask)
            
            emb_t = self.emb_dec(fab_input)
            
            fab_decoder_output, (fab_hidden, fab_cell) = self.decoder(emb_t,
                                                                      fab_h_t_tilde,
                                                                      (fab_hidden, fab_cell))
            context_vector = self.attn(fab_h_src, fab_decoder_output, fab_mask)
            fab_h_t_tilde = self.tanh(self.concat(torch.cat([fab_decoder_output,
                                                             context_vector
                                                             ], dim=-1)))
            y_hat = self.generator(fab_h_t_tilde)
            
            cnt = 0
            for space in spaces:
                if space.is_done() == 0:
                    from_index = cnt * beam_size
                    to_index = from_index + beam_size
                    
                    space.collect_result(y_hat[from_index:to_index],
                                         (fab_hidden[:, from_index:to_index, :],
                                          fab_cell[:, from_index:to_index, :]
                                          ),
                                         fab_h_t_tilde[from_index:to_index])
                    cnt += 1
                
            done_cnt = [space.is_done() for space in spaces]
            length += 1
        
        batch_sentences = []
        batch_probs = []
        
        for i, space in enumerate(space):
            sentences, probs = space.get_n_best(n_best)
            
            batch_sentences += [sentences]
            batch_probs += [probs]
            
        return batch_sentences, batch_probs

## 10.7 성능 평가

### 10.7.1 정성적 평가

실제 사람이 블라인드 테스트

### 10.7.2 정량적 평가

#### PPL

- 신경망 기계번역도 조건부 언어모델이므로 퍼플렉서티를 통해 측정 가능
- 교차 엔트로피 손실값에 exp로 구함

#### BLEU

- PPL이 실제 번역기 성능과 완벽한 비례 X
- 정답 문장과 예측 문장 사이에 일치하는 n-gram 개수의 비율의 기하평균에 따라 계산

$$BLEU = brevity-penalty * \prod_{n=1}^N p_n^{w_n}$$
$$brevity-penalty = min(1, \frac {|prediction|} {|reference|})$$
$$p_n \ is \ precision \ of \ n-gram \ and \ w_n=(\frac 1 2)^n$$

In [20]:
0.5**0.25

0.8408964152537145